In [1]:
%matplotlib widget
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import seaborn as sns
import pandas as pd
from pathlib import Path
data_dir = Path('data/Torus')

In [2]:
sns.set_style("whitegrid")


In [3]:
from src.modules import Evolve, Torus, Ring, utils, Plotter, metric

## Nivel de Consenso según el aumento de inestables
## Grilla de 16 x 16      |       3000 pasos        |           400 muestras

### Load

In [4]:
n=20
title = 'Torus_agreement_{0}.csv'.format(n)
data= pd.read_csv(data_dir /title)

In [6]:
data['runstab']=data.unstables/(400-data.unstables)

In [7]:
data

,init_magnet,length,Consensus_mean,Magnetization_min,Magnetization_max,Magnetization_std,Energy,unstables,runstab
0,0.0,1,0.005,0.005,0.005,0.0,-0.625,0,0.000000
1,0.0,1,0.005,0.005,0.005,0.0,-0.625,1,0.002506
2,0.0,1,0.005,0.005,0.005,0.0,-0.625,2,0.005025
3,0.0,1,0.005,0.005,0.005,0.0,-0.625,3,0.007557
4,0.0,1,0.005,0.005,0.005,0.0,-0.625,4,0.010101
...,...,...,...,...,...,...,...,...,...
205356,0.6,1,0.980,0.980,0.980,0.0,-0.980,65,0.194030
205357,0.6,1,0.980,0.980,0.980,0.0,-0.980,66,0.197605
205358,0.6,1,0.980,0.980,0.980,0.0,-0.980,67,0.201201
205359,0.6,1,1.000,1.000,1.000,0.0,-1.000,68,0.204819


In [25]:
data = data[data.runstab!=np.inf]

AttributeError: 'DataFrame' object has no attribute 'runstab'

In [29]:
data

,init_magnet,length,Consensus_mean,Magnetization_min,Magnetization_max,Magnetization_std,Energy,unstables,runstab
0,0.0,1,0.005,0.005,0.005,0.0,-0.625,0,0.000000
1,0.0,1,0.005,0.005,0.005,0.0,-0.625,1,0.002506
2,0.0,1,0.005,0.005,0.005,0.0,-0.625,2,0.005025
3,0.0,1,0.005,0.005,0.005,0.0,-0.625,3,0.007557
4,0.0,1,0.005,0.005,0.005,0.0,-0.625,4,0.010101
...,...,...,...,...,...,...,...,...,...
205356,0.6,1,0.980,0.980,0.980,0.0,-0.980,65,0.194030
205357,0.6,1,0.980,0.980,0.980,0.0,-0.980,66,0.197605
205358,0.6,1,0.980,0.980,0.980,0.0,-0.980,67,0.201201
205359,0.6,1,1.000,1.000,1.000,0.0,-1.000,68,0.204819


In [8]:
df = data.query('init_magnet <=0.4')

In [9]:
fig, ax = plt.subplots()
sns.lineplot(data=df, x='unstables', y='Consensus_mean', ci=None, hue='init_magnet')
plt.legend(bbox_to_anchor=(0.85, 1.1), loc=2, borderaxespad=0.)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plt.savefig('data/torus_agreement_uns.svg', format='svg', transparent=True)

In [34]:
fig, ax = plt.subplots()
sns.lineplot(data=df, x='runstab', y='Consensus_mean', ci=None, hue='init_magnet')
plt.legend(bbox_to_anchor=(0.85, 1.1), loc=2, borderaxespad=0.)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
plt.savefig('data/torus_agreement.runs.svg', format='svg', transparent=True)

In [7]:
facet = sns.FacetGrid(data,  # the dataframe to pull from
                          row="init_magnet",  # define the column for each subplot row to be differentiated by
                          hue="init_magnet",  # define the column for each subplot color to be differentiated by
                          aspect=6,  # aspect * height = width
                          height=2,  # height of each subplot
                          )
by_magnet = data.groupby("init_magnet")
def init_magnet(x, color, label):
    ax= plt.gca()
    fix_magnet = by_magnet.get_group(float(label))
    val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["Consensus_mean"].values)
    print(label, val)
    plt.axhline(y=val, ls="--")
    
facet.map(init_magnet, "init_magnet")
facet.map(sns.lineplot,'unstables',"Consensus_mean")
#facet.map(sns.lineplot, 'unstables', "Magnetization_min", ls="--")
#facet.map(sns.lineplot, 'unstables', "Magnetization_max", ls= "--")

def label(x, color, label):
    ax = plt.gca()  # get the axes of the current object
    ax.text(0, .2,  # location of text
            label,  # text label
            fontweight="bold", color=color, size=18,  # text attributes
            ha="left", va="center",  # alignment specifications
            transform=ax.transAxes)  # specify axes of transformation
        

facet.map(label, "Consensus_mean")  # the function counts as a plotting object!
facet.set_titles("")  # set title to blank

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0 0.07194871794871796
0.2 0.48128205128205126
0.4 0.8182307692307693
0.6 0.9621538461538461
0.8 0.9973846153846154


## Nivel de Consenso según el aumento de inestables
## Grilla de 18 x 18      |       3500 pasos        |           400 muestras

### Load

In [7]:
n=18
title = 'Torus_agreement_{0}.csv'.format(n)
data= pd.read_csv(data_dir /title)

In [8]:
facet = sns.FacetGrid(data,  # the dataframe to pull from
                          row="init_magnet",  # define the column for each subplot row to be differentiated by
                          hue="init_magnet",  # define the column for each subplot color to be differentiated by
                          aspect=6,  # aspect * height = width
                          height=2,  # height of each subplot
                          )
by_magnet = data.groupby("init_magnet")
def init_magnet(x, color, label):
    ax= plt.gca()
    fix_magnet = by_magnet.get_group(float(label))
    val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["Consensus_mean"].values)
    print(label, val)
    plt.axhline(y=val, ls="--")
    
facet.map(init_magnet, "init_magnet")
facet.map(sns.lineplot,'unstables',"Consensus_mean")
#facet.map(sns.lineplot, 'unstables', "Magnetization_min", ls="--")
#facet.map(sns.lineplot, 'unstables', "Magnetization_max", ls= "--")

def label(x, color, label):
    ax = plt.gca()  # get the axes of the current object
    ax.text(0, .2,  # location of text
            label,  # text label
            fontweight="bold", color=color, size=18,  # text attributes
            ha="left", va="center",  # alignment specifications
            transform=ax.transAxes)  # specify axes of transformation
        

facet.map(label, "Consensus_mean")  # the function counts as a plotting object!
facet.set_titles("")  # set title to blank

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0 0.07271604938271604
0.2 0.4860956790123456
0.4 0.8142901234567901
0.6 0.9622376543209876
0.8 0.9978395061728395


## Decaimiento de energía al agregar inestabilidad

In [9]:
facet = sns.FacetGrid(data,  # the dataframe to pull from
                          row="init_magnet",  # define the column for each subplot row to be differentiated by
                          hue="init_magnet",  # define the column for each subplot color to be differentiated by
                          aspect=6,  # aspect * height = width
                          height=2,  # height of each subplot
                          )
def init_magnet(x, color, label):
    ax= plt.gca()
    fix_magnet = by_magnet.get_group(float(label))
    val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["Energy"].values)
    print(label, val)
    plt.axhline(y=val, ls="--")
    
facet.map(init_magnet, "init_magnet")
facet.map(sns.lineplot,'unstables',"Energy")
#facet.map(sns.lineplot, 'unstables', "Magnetization_min", ls="--")
#facet.map(sns.lineplot, 'unstables', "Magnetization_max", ls= "--")

def label(x, color, label):
    ax = plt.gca()  # get the axes of the current object
    ax.text(0, .2,  # location of text
            label,  # text label
            fontweight="bold", color=color, size=18,  # text attributes
            ha="left", va="center",  # alignment specifications
            transform=ax.transAxes)  # specify axes of transformation
        

facet.map(label, "Energy")  # the function counts as a plotting object!
facet.set_titles("")  # set title to blank

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0 -0.6005092592592592
0.2 -0.6975308641975309
0.4 -0.8582407407407407
0.6 -0.9663734567901235
0.8 -0.9979012345679013


In [10]:
by_magnet = data.groupby("init_magnet")

In [12]:
fix_magnet = by_magnet.get_group(0.4)
val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["Consensus_mean"].values)

vow = fix_magnet[ fix_magnet['unstables'] == 0]['init_magnet'].count()
vow

400

## Nivel de Consenso según el aumento de inestables
## Grilla de 20 x 20      |       5000 pasos        |           400 muestras

### Load

In [16]:
n=20
title = 'Torus_agreement_{0}.csv'.format(n)
data= pd.read_csv(data_dir /title)

In [17]:
facet = sns.FacetGrid(data,  # the dataframe to pull from
                          row="init_magnet",  # define the column for each subplot row to be differentiated by
                          hue="init_magnet",  # define the column for each subplot color to be differentiated by
                          aspect=6,  # aspect * height = width
                          height=2,  # height of each subplot
                          )
by_magnet = data.groupby("init_magnet")
def init_magnet(x, color, label):
    ax= plt.gca()
    fix_magnet = by_magnet.get_group(float(label))
    val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["Consensus_mean"].values)
    print(label, val)
    plt.axhline(y=val, ls="--")
    
facet.map(init_magnet, "init_magnet")
facet.map(sns.lineplot,'unstables',"Consensus_mean")
#facet.map(sns.lineplot, 'unstables', "Magnetization_min", ls="--")
#facet.map(sns.lineplot, 'unstables', "Magnetization_max", ls= "--")

def label(x, color, label):
    ax = plt.gca()  # get the axes of the current object
    ax.text(0, .2,  # location of text
            label,  # text label
            fontweight="bold", color=color, size=18,  # text attributes
            ha="left", va="center",  # alignment specifications
            transform=ax.transAxes)  # specify axes of transformation
        

facet.map(label, "Consensus_mean")  # the function counts as a plotting object!
facet.set_titles("")  # set title to blank

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0 0.07194871794871796
0.2 0.48128205128205126
0.4 0.8182307692307693
0.6 0.9621538461538461
0.8 0.9973846153846154


## Decaimiento de energía al agregar inestabilidad

In [15]:
facet = sns.FacetGrid(data,  # the dataframe to pull from
                          row="init_magnet",  # define the column for each subplot row to be differentiated by
                          hue="init_magnet",  # define the column for each subplot color to be differentiated by
                          aspect=6,  # aspect * height = width
                          height=2,  # height of each subplot
                          )
def init_magnet(x, color, label):
    ax= plt.gca()
    fix_magnet = by_magnet.get_group(float(label))
    val = np.mean(fix_magnet.loc[fix_magnet["unstables"] == 0]["Energy"].values)
    print(label, val)
    plt.axhline(y=val, ls="--")
    
facet.map(init_magnet, "init_magnet")
facet.map(sns.lineplot,'unstables',"Energy")
#facet.map(sns.lineplot, 'unstables', "Magnetization_min", ls="--")
#facet.map(sns.lineplot, 'unstables', "Magnetization_max", ls= "--")

def label(x, color, label):
    ax = plt.gca()  # get the axes of the current object
    ax.text(0, .2,  # location of text
            label,  # text label
            fontweight="bold", color=color, size=18,  # text attributes
            ha="left", va="center",  # alignment specifications
            transform=ax.transAxes)  # specify axes of transformation
        

facet.map(label, "Energy")  # the function counts as a plotting object!
facet.set_titles("")  # set title to blank

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0 -0.6042307692307692
0.2 -0.692923076923077
0.4 -0.8611794871794872
0.6 -0.9656153846153847
0.8 -0.9975641025641026
